In [ ]:
from glob import glob
import json
import os
import pandas as pd

import pypowsybl as pp

from IPython.core.display import display,SVG,HTML

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objects as go

from dash_canvas import DashCanvas

from jupyter_dash import JupyterDash

In [ ]:
# write width and height attributes in SDL's SVG
def fixSvg(svgs, width=700, height=700):
    SVG_HEADER1 = '<svg xmlns=\"http://www.w3.org/2000/svg\">'
    SVG_HEADER2 = '<svg xmlns=\"http://www.w3.org/2000/svg\" width=\"{w:}px\" height=\"{h:}px\" >'
    return svgs.replace(SVG_HEADER1, SVG_HEADER2.format(w=width, h=height))


In [ ]:
# save the SVG string to FS
def saveSvg(svg_string, filename):
    svgFile = open(filename, "wt")
    n = svgFile.write(svg_string)
    svgFile.close() 

In [ ]:
# get a network
n = pp.network.create_four_substations_node_breaker_network()

subId = 'S1'
vlid = 'S1VL1'

In [ ]:
# write the SVGs to the asset folders (directory served by dash, by default)
vlSvg1 = fixSvg(str(n.get_single_line_diagram(vlid)), width=800, height=800);
saveSvg(vlSvg1, 'assets/vl1_1.svg')

subSvg2 = fixSvg(str(n.get_single_line_diagram(subId)), width=1700, height=800);
saveSvg(subSvg2, 'assets/sub2_1.svg')

filenames = sorted(glob(os.path.join('assets', '*.svg')))



In [ ]:
#show SVG - DashCanvas and Jupyter-dash

app = JupyterDash(__name__)

app.layout = html.Div([
        html.Div([
            dcc.Dropdown(
            id='filenames-dropdown2', clearable=False,
            value=filenames[1], options=[{'label': c, 'value': c}
                                         for c in filenames])
        ], className="three columns"),
        html.Div([
        DashCanvas(id='canvas_image',
                hide_buttons=['line', 'pencil', 'rectangle', 'undo', 'redo', 'select', 'Save'],
                tool='pan',
                width=900,
                height=850)
        ], className="cust-dash-canvas")
        ])

@app.callback(
    Output('canvas_image', 'image_content'),
    [Input("filenames-dropdown2", "value")]
)
def update_canvas_filename(value):
    return value

app.run_server(mode='inline', height='600')